# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""14336737299303…","""../../../../da…",1.2927e19,"""69217977927239…","""13610174356200…","""13002889636067…",false
"""16306038951534…","""../../../../da…",1.4856e19,"""34981408658821…","""13610174356200…","""15957769775016…",true
"""16325995777112…","""../../../../da…",5.6458e17,"""13663255640684…","""49720179974435…","""13002889636067…",false
"""54164188308987…","""../../../../da…",4.0723e18,"""34981408658821…","""79099256380439…","""13002889636067…",false
"""51155595641267…","""../../../../da…",1.4677e19,"""69217977927239…","""79099256380439…","""52786108153847…",true
"""11399637419097…","""../../../../da…",1.6330e19,"""13663255640684…","""18092779239219…",null,false
"""51230081897379…","""../../../../da…",8.7675e18,"""13663255640684…","""79099256380439…","""52786108153847…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""16606175969325…","""../../../../da…",1.2927e19,"""86050135458531…","""FirstMortgage3…","""52158912173824…","""Rejected"""
"""48989449425371…","""../../../../da…",1.4856e19,"""53802500056944…","""FirstMortgage3…","""40005535483459…","""Accepted"""
"""73936259108684…","""../../../../da…",5.6458e17,"""11416644921693…","""MoneyMarketSav…","""52158912173824…","""Rejected"""
"""41218805354321…","""../../../../da…",4.0723e18,"""53802500056944…","""BasicChecking""","""52158912173824…","""Rejected"""
"""46153581794986…","""../../../../da…",1.4677e19,"""86050135458531…","""BasicChecking""","""60474798080983…","""Accepted"""
"""70232526135233…","""../../../../da…",1.6330e19,"""11416644921693…","""UPlusFinPerson…",null,"""Rejected"""
"""50809932004242…","""../../../../da…",8.7675e18,"""11416644921693…","""BasicChecking""","""60474798080983…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""13900848103644…","""../../../../da…",1.2927e19,"""10915645239905…","""FirstMortgage3…","""82981218542419…","""Rejected"""
"""79855234553166…","""../../../../da…",1.4856e19,"""12025900207842…","""FirstMortgage3…","""55031406841984…","""Accepted"""
"""11915403097034…","""../../../../da…",5.6458e17,"""30855713155062…","""MoneyMarketSav…","""82981218542419…","""Rejected"""
"""11681093620698…","""../../../../da…",4.0723e18,"""12025900207842…","""BasicChecking""","""82981218542419…","""Rejected"""
"""43179915290136…","""../../../../da…",1.4677e19,"""10915645239905…","""BasicChecking""","""34207304308224…","""Accepted"""
"""81483321853145…","""../../../../da…",1.6330e19,"""30855713155062…","""UPlusFinPerson…",null,"""Rejected"""
"""51471146324585…","""../../../../da…",8.7675e18,"""30855713155062…","""BasicChecking""","""34207304308224…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""14036233056649…",1.2927e19,"""12910020278855…","""FirstMortgage3…","""68848262557517…","""Rejected"""
"""10482833503038…",1.4856e19,"""92568337079192…","""FirstMortgage3…","""16297231089045…","""Accepted"""
"""71999243453110…",5.6458e17,"""21317888133778…","""MoneyMarketSav…","""68848262557517…","""Rejected"""
"""10374619281367…",4.0723e18,"""92568337079192…","""BasicChecking""","""68848262557517…","""Rejected"""
"""88595475598485…",1.4677e19,"""12910020278855…","""BasicChecking""","""12678878702894…","""Accepted"""
"""16624926707498…",1.6330e19,"""21317888133778…","""UPlusFinPerson…",null,"""Rejected"""
"""10631274710588…",8.7675e18,"""21317888133778…","""BasicChecking""","""12678878702894…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'